-----------------
# TensorFlow: Shapes and Dynamic Dimensions

-----------------
Here is a simple How-To to understand the concept of shapes in TensorFlow and hopefully avoid losing hours of time debugging them

------------------

-----------------
## What Is A Tensor Again?

-----------------
- Very briefly, a tensor is an N-dimensional array containing the same type of data (int32, bool, etc.)
    - All you need to describe a tensor fully is its **data type** and the **value of each of the N dimension**
    - That’s why we describe a tensor with what we call a **shape**: 
        - The shape is a **list, tuple or TensorShape** of numbers containing the size (length) of each dimension of our tensor

-----------------
***NOTE: FOR NOMENCLATURE PURPOSES WHEN DESCRIBING SHAPE WE WILL BE USING THE LABELS: D, W, and H (Depth, Width, and Height)***

-----------------

**Let's look at a tensor briefly before moving on...**

--------------

In [ ]:
import tensorflow as tf

my_tensor = tf.constant(0., shape=[6,3,7])    #  D == 6,  W == 3,  H == 7   i.e. This is a matrix/tensor

print(my_tensor) # Tensor("Const:0", shape=(6, 3, 7), dtype=float32)

-----------------
**Note The Following In The Cell Above**

-----------------
**1.** It has a **name** used in a key-value store to retrieve it later: ***Const_#:0*** *(# may change if the cell is re-run)*  

**2.** It has a **shape** describing the size of each dimension: ***(6, 3, 7)*** 

**3.** It has a **type**: ***float32***

-----------------

-------------
## Why Are We Looking Into Tensor Shapes ??

-------------

### Because Tensorflow is special when it comes to shapes and that is what we wish to explore in this how-to

-------------

The meat of this tutorial is explaining that Tensorflow Has **TWO types of shapes** --- **<font color=red>Dynamic</font>** and **<font color=green>Static</font>** **<font color=pink>!</font><font color=blue>!</font><font color=cyan>!</font><font color=yellow>!</font><font color=orange>!</font>** 

-------------

-----------
# <font color=green>Tensors Having Static Shape</font>

-----------

The static shape is the shape you provided when creating a tensor...

                                                            OR

The static shape is the shape inferred by TensorFlow when you define an operation resulting in a new tensor

-----------------

                                            The static shape is a tuple or a list

-----------------

TensorFlow will do its best to guess the shape of your different tensors (between your different operations) but it won’t always be able to do it. Especially if you start to do operations with placeholders defined with unknown dimensions (like when you want to use a dynamic batch size)

-----------------

To use the static shape (accessing/changing) in your code:
- You will use the different functions which are attached to the **TENSOR ITSELF**
    - These functions **DO HAVE** an underscore in their names
    
-----------------

**<font color=green>Let's look at an example illustrating tensors with static shapes:</font>**

----------------

In [ ]:
# Import Tensorflow (Even though we did it above, let's just do it so we can keep all the code in one cell)
import tensorflow as tf

# To clear the defined variables and operations of all the previous cells
tf.reset_default_graph()

print("---------------------------------------------------------------------------------\n")

# First Tensor with a defined shape
my_tensor = tf.constant(0., shape=[6,2])

# Access the shape property of my_tensor and save it to my_static_shape
my_static_shape = my_tensor.get_shape()

# Print the data type of the shape property of my_tensor 

# Full description: TensorShape([Dimension(6), Dimension(2)])
print("The data type of the shape property of my_tensor is : {}".format(type(my_static_shape)))   

# Shape as we usually know it: (6, 2)
print("The shape of my_tensor as a tuple : {}".format(my_static_shape))      


# If you want to convert the shape to a list you can do that with the .as_list() command

# Shape Turned From Tuple to List:   [6, 2]
print("The shape of my_tensor as a list : {}\n".format(my_static_shape.as_list()))

print("---------------------------------------------------------------------------------\n")

# Let's do an operation resulting in a new tensor to see what happens (tf.transpose makes rows into cols and cols into rows)
my_tensor_transposed = tf.transpose(my_tensor)

# Print the shape of the new tensor that was generated from the transpose operation

# Shape as we usually know it: (2, 6)
print("The shape of my_tensor_transposed as a tuple : {}".format(my_tensor_transposed.get_shape()))

# The static shape has been inferred by TensorFlow based on the transpose operation and the tensors used by the operation

# Let's take a look at the static shape and double check
print("The data type of the shape property of my_tensor_transposed is : {}\n".format(type(my_tensor_transposed.get_shape())))

print("---------------------------------------------------------------------------------\n")

# At any moment, you can also force a Tensor with an undefined (None) dimension to have a precise shape of the same rank
# This might be required if the shape can't be inferred from the graph alone (dynamic input data...)

# Let's see what an undefined dimension looks like by using a placeholder (constant can't have None type as dimension)
my_placeholder = tf.placeholder(tf.float32, shape=[None, 2])

# Print the shape of my_placeholder (tuple)

# Shape as we usually know it: (?, 2)
print("The shape of my_placeholder as a tuple : {}".format(my_placeholder.get_shape()))

# Now let's lock the undefined shape in to a specific value (None --> 8) using .set_shape([])
my_placeholder.set_shape([8, 2])

# Now let's view the new shape of my_placeholder (tuple)

 # Shape as we usually know it: (8, 2)
print("The shape of my_placeholder as a tuple after having shape locked in : {}\n".format(my_placeholder.get_shape()))

print("---------------------------------------------------------------------------------")

-----------

***Note: The static shape is very useful to debug your code with print so you can check your tensors have the right shapes***

-----------

-----------
# <font color=red>Tensors Having Dynamic Shape</font>

-----------

--------------
The dynamic shape is will actually be in use when we run a graph (sess.run)

--------------
- **The dynamic shape is a tensor that describes the shape of the original tensor (i.e. it is NOT the original tensor)**
    - If you defined a placeholder with undefined dimensions (with the None type as a dimension), those None dimensions will only have a real value when you feed an input to your placeholder
    
--------------


To use the dynamic shape (accessing/changing) in your code:
- You will use the different functions which are attached to the **MAIN SCOPE**
    - These functions **DON'T HAVE** an underscore in their names
    
-----------------

**<font color=red>Let's look at an example illustrating tensors with dynamic shapes:</font>**

----------------

In [21]:
# Import Tensorflow (Even though we did it above, let's just do it so we can keep all the code in one cell)
import tensorflow as tf

# To clear the defined variables and operations of all the previous cells
tf.reset_default_graph()

print("---------------------------------------------------------------------------------\n")

# First Tensor with a defined shape (note I used placeholder to show of dynamic changability later-on)
my_tensor = tf.placeholder(tf.float32, shape=[None ,2]) # Tensor('Const:0' shape=(6, 2) dtype=int32)

# Let's create a new tensor that represents the shape of my_tensor using the command tf.shape(my_tensor)
# NOTE: tf.shape(my_tensor) is different than my_tensor.get_shape (link in cell below)
my_dynamic_shape = tf.shape(my_tensor) 

# Let's print the dynamic shape of my_dynamic_shape, the tensor representing the shape of the tensor my_tensor
# Tensor('Shape:0' shape=(2,) dtype=int32)
print("This is the shape (dynamic) of the my_dynamic_shape, the tensor representing the shape of my_tensor (static) : \n{}\n".format(my_dynamic_shape))


# The shape of the tensor "Shape" is (2,) because my_tensor is a 2-D tensor
# The dynamic shape is a 1-D tensor containing sizes of my_tensor dimensions
# In this case, we have 2 dimensions... hence the shape being (2,)

# Let's reshape the original tensor (my_tensor) into a 3-D tensor (as opposed to a 2-D tensor)
my_reshaped_tensor = tf.reshape(my_tensor, [2, 3, 2]) 

# Let's look at the dynamic shape of the reshaped tensor (it should have shape=(3,) as the reshaped tensor is now 3-D)
print("This is the shape (dynamic) tensor representing the shape of my_reshaped_tensor (static) : \n{}\n".format(tf.shape(my_reshaped_tensor)))

# To access a dynamic shape value, you need to run your graph and feed any placeholder that your tensor my depended upon:
with tf.Session() as sess:
    
    # Define feed_dictionary for the my_tensor (must be of the required shape -- 2D but can be any length)
    
    # Here is one where feed dictionary has a length of 7
    dictionary_vals_7_2 = [[1.,2.], [1., 2.], [1., 2.], [1., 2.], [1., 2.], [1., 2.], [1., 2.]]
    print("Here is my_dynamic_shape if we define it using a dictionary with   7 pairs  :  {}".format(my_dynamic_shape.eval(feed_dict={my_tensor: dictionary_vals_7_2})))
    
    # Here is one where feed dictionary has a length of 2
    dictionary_vals_2_2 = [[1., 2.], [1., 2.]]
    print("Here is my_dynamic_shape if we define it using a dictionary with   2 pairs  :  {}".format(my_dynamic_shape.eval(feed_dict={my_tensor: dictionary_vals_2_2})))
    
    # Here is one where feed dictionary has a length of 1
    dictionary_vals_1_2 = [[1.,2.]]
    print("Here is my_dynamic_shape if we define it using a dictionary with   1 pairs  :  {}".format(my_dynamic_shape.eval(feed_dict={my_tensor: dictionary_vals_1_2})))
    

---------------------------------------------------------------------------------

This is the shape (dynamic) of the my_dynamic_shape, the tensor representing the shape of my_tensor (static) : 
Tensor("Shape:0", shape=(2,), dtype=int32)

This is the shape (dynamic) tensor representing the shape of my_reshaped_tensor (static) : 
Tensor("Shape_1:0", shape=(3,), dtype=int32)

Here is my_dynamic_shape if we define it using a dictionary with   7 pairs  :  [7 2]
Here is my_dynamic_shape if we define it using a dictionary with   2 pairs  :  [2 2]
Here is my_dynamic_shape if we define it using a dictionary with   1 pairs  :  [1 2]


----------
Click For Help If Necessary -- [Clarification Of Difference Between tf.shape(tensor) and tensor.get_shape From Stack Overflow](https://stackoverflow.com/questions/37096225/how-to-understand-static-shape-and-dynamic-shape-in-tensorflow)

---------

***The dynamic shape is very handy for dealing with dimensions that you want to keep dynamic***

-------------